In [2]:
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import CategoricalNB

In [3]:
def create_datasets():
  Outlook = ['Rainy', 'Sunny', 'Overcast', 'Overcast', 'Sunny', 'Rainy',
            'Sunny', 'Overcast', 'Rainy', 'Sunny', 'Sunny', 'Rainy',
            'Overcast', 'Overcast']
  Play = ['Yes', 'Yes', 'Yes', 'Yes',
          'No', 'Yes', 'Yes', 'Yes', 
          'No', 'No', 'Yes', 'No','Yes', 'Yes']

  df1 = pd.DataFrame(data={'Outlook': Outlook,
                          'Play': Play})
  Outlook = ['Rainy','Rainy', 'Overcast', 'Sunny', 'Sunny', 'Sunny', 'Overcast', 
            'Rainy', 'Rainy','Sunny', 'Rainy', 'Overcast', 'Overcast', 'Sunny']
  Temp = ['Hot', 'Hot','Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild',
          'Cool', 'Mild','Mild', 'Mild', 'Hot', 'Mild', ]
  Humidity = ['High', 'High','High', 'High', 'Normal', 'Normal', 'Normal', 'High',
          'Normal', 'Normal','Normal', 'High', 'Normal', 'High', ]
  Windy = ['False', 'True','False', 'False', 'False', 'True', 'True', 'False',
          'False', 'False','True', 'True', 'False', 'True', ]
  play =['No', 'No','Yes', 'Yes', 'Yes', 'No', 'Yes', 'No',
          'Yes', 'Yes','Yes', 'Yes', 'Yes', 'No']

  df2 = pd.DataFrame(data={'Outlook': Outlook,
                          'Temp': Temp,
                          'Humidity': Humidity,
                          'Windy': Windy,
                          'play': play})
  return df1, df2                  

In [4]:
#TODO create set to pass predict data as strings, not int
class naive():

  def fit(self, df, target_column):
    
    assert isinstance(df, pd.DataFrame) or isinstance(df, np.ndarray), f'input data should be DataFrame or np array, got {type(self.df)}'
    self.dtype = type(df)

    if self.dtype == pd.DataFrame:
      self.target_names = df[target_column].unique()
      
      for count, dtype in enumerate(df.dtypes):
        if dtype != 'int64':
          column = df.columns[count]
          df[column].replace(df[column].unique(), range(len(df[column].unique())), inplace=True)

      self.df_to_np = np.concatenate([df.drop(columns=target_column).values.reshape(len(df), len(df.columns) - 1), 
                  df[target_column].values.reshape(len(df[target_column]), 1)], axis=1)
    else:
      self.df_to_np = df
 
  def print_data(self):
    print(df)
    print(self.df_to_np.shape[2])


  def predict(self, to_predict):
    independent_cols = self.df_to_np[:, 0:-1]
    dependent_col = self.df_to_np[:, -1]
    #to correct
    results = []
    for target in np.unique(dependent_col):
      result = 0.0
      p_xys = []
      p_xs = []
      for i in range(self.df_to_np.shape[1] - 1):
        independent_col = independent_cols[:, i]

        number_of_x_in_y = np.count_nonzero(independent_col == to_predict[i])

        p_x = number_of_x_in_y / len(independent_col)
        p_y = np.count_nonzero(dependent_col == target) / len(independent_col)
        p_xy = np.sum((independent_col == to_predict[i]) & (dependent_col == target)) / np.sum(dependent_col == target)

        #print(f'p_x: {p_x}, p_xy:{p_xy}, p_y:{p_y}')
        p_xys.append(p_xy)
        p_xs.append(p_x)
      #print(p_xys, p_y)
      result = (np.prod(p_xys) * p_y) / np.prod(p_xs) 
      results.append(result)

      if self.dtype == pd.DataFrame:
        print(f'value for {self.target_names[target]}: {result:.5f}')
      else:
        print(f'value for {target}: {result:.5f}')
    for count, target in enumerate(np.unique(dependent_col)):
      if self.dtype == pd.DataFrame:
        print(f'probability for {self.target_names[target]} : {results[count] / sum(results):.5f}')
      else:
        print(f'probability for {target} : {results[count] / sum(results):.5f}')

In [5]:
df1, df2 = create_datasets()

In [6]:
na = naive()
na.fit(df1, 'Play')
na.predict([1])

value for Yes: 0.60000
value for No: 0.40000
probability for Yes : 0.60000
probability for No : 0.40000


In [7]:
na = naive()
na.fit(df2, 'play')
#na.print_data()
#Rainy Cool High True ?
na.predict([0,2, 0,1])

value for No: 0.94080
value for Yes: 0.24198
probability for No : 0.79542
probability for Yes : 0.20458


In [8]:
na = naive()
na.fit(np.array(df2), 'play')
#na.print_data()
#Rainy Cool High True ?
na.predict([0,2, 0,1])

value for 0: 0.94080
value for 1: 0.24198
probability for 0 : 0.79542
probability for 1 : 0.20458
